In [1]:
import pandas as pd 
from google.colab import drive 
from random import randint
from PIL import Image, ImageDraw
from math import ceil, floor

In [2]:
model_predictions_path = '/content/gdrive/My Drive/simpsons/output.csv'
annotation_loc = '/content/gdrive/My Drive/simpsons/annotation.txt'

In [3]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
def intersection_over_union(boxA, boxB):
  xA = max(boxA[0], boxB[0])
  yA = max(boxA[1], boxB[1])
  xB = min(boxA[2], boxB[2])
  yB = min(boxA[3], boxB[3])
  
  interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
  boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
  boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
  return (interArea / float(boxAArea + boxBArea - interArea))
  
def convert_bbox(old_size, new_size, x_min, y_min, x_max, y_max):
  x_min_new = floor((x_min / old_size[0]) * new_size[0])
  y_min_new = floor((y_min / old_size[1]) * new_size[1])
  x_max_new = ceil((x_max / old_size[0]) * new_size[0])
  y_max_new = ceil((y_max / old_size[1]) * new_size[1])
  
  return [x_min_new, y_min_new, x_max_new, y_max_new]

In [5]:
results = pd.read_csv(model_predictions_path)
annotations = pd.read_csv(annotation_loc, header = None)

In [6]:
file_names = results["file_name"].to_list()
predicted_classes = results["classes"].to_list()

In [7]:
ious = []
correct = 0
idx_to_sample = []
for _ in range(30):
	idx_to_sample.append(randint(0, len(file_names)))

sample_output_dir = '/content/gdrive/My Drive/simpsons/od_images/'
 
for i in range(len(file_names)):
  category = file_names[i].split('/')[-2]
  if category == predicted_classes[i]:
    correct += 1
  annotation_for_i = annotations[annotations[0] == file_names[i]]

  im = Image.open(file_names[i])
  org_shape = im.size

  gt_box = []
  gt_box.append(annotation_for_i.iloc[0][1])
  gt_box.append(annotation_for_i.iloc[0][2])
  gt_box.append(annotation_for_i.iloc[0][3])
  gt_box.append(annotation_for_i.iloc[0][4])

  predicted_box = []
  predicted_box.append(results.iloc[i][2])
  predicted_box.append(results.iloc[i][3])
  predicted_box.append(results.iloc[i][4])
  predicted_box.append(results.iloc[i][5])

  predicted_box = convert_bbox((300,300), 
                               org_shape,
                               predicted_box[0],
                               predicted_box[1],
                               predicted_box[2],
                               predicted_box[3],
                               )

  iou = intersection_over_union(gt_box, predicted_box)
  ious.append(iou)

  if i in idx_to_sample:
    draw = ImageDraw.Draw(im)
    draw.rectangle(gt_box, outline="red", width=3)
    draw.rectangle(predicted_box, outline="green", width=3)
    draw.text((10,10), text = predicted_classes[i], fill = "green")
    im.save(sample_output_dir + category + "_" + file_names[i].split('/')[-1])


print("Class prediction accuracy: {:.2f}".format(correct/len(predicted_classes)* 100))
print("Mean IOU: {:.2f}". format(sum(ious) / len(ious)))

Class prediction accuracy: 89.52
Mean IOU: 0.68
